# Avantis Collections

## SQL query 

In [ ]:
import pyodbc
import pandas as pd
import os
import re
from sqlalchemy.engine import URL
from sqlalchemy import create_engine

connect = 'DSN=' + os.environ['Avantis_Server'] + ';UID='+ os.environ['Avantis_User'] + ';PWD=' + os.environ['Avantis_Pass']
connection_url = URL.create("mssql+pyodbc", query={"odbc_connect": connect})

engine = create_engine(connection_url)

SQL1 = """
SELECT DISTINCT
	PMJOB1.id AS PM_JOB, 
	PMTASK1.tskid, 
	PMTASK1.aenm AS PM_TASK_NAME, 
	MAINTENT1.id AS TASK_ENTITY,
    MAINTENT1.aenm AS TASK_ENTITY_NAME,
    MTNTYPE1.id AS TASK_ENTITY_TYPE, 
	PMTLACTIVITY1.actseq, 
	MAINTENT2.id AS ACT_ENTITY,
    ENTCLASS1.entclsid AS ACT_CLASS,
    PMTLACTIVITY1.actdesc

FROM            
    mc.MAINTENT AS MAINTENT1 INNER JOIN
    mc.SITE ON MAINTENT1.BEparn_oi = mc.SITE.siteoi FULL OUTER JOIN
    mc.MTNTYPE MTNTYPE1 ON MAINTENT1.mtctyp_oi = MTNTYPE1.mtoi FULL OUTER JOIN
    mc.MAINTENT AS MAINTENT2 FULL OUTER JOIN
    mc.ENTCLASS ENTCLASS1 ON MAINTENT2.entclsref_oi = ENTCLASS1.entcloi FULL OUTER JOIN
    mc.PMTLACTIVITY AS PMTLACTIVITY1 RIGHT OUTER JOIN
    mc.TSKLNK AS TSKLNK1 ON PMTLACTIVITY1.pmtlr_oi = TSKLNK1.pmtloi LEFT OUTER JOIN
    mc.ACTIVITYTYPE AS ACTIVITYTYPE1 ON PMTLACTIVITY1.acttype_oi = ACTIVITYTYPE1.acttpoi LEFT OUTER JOIN
    mc.pmtl_freq AS pmtl_freq1 ON TSKLNK1.pmtloi = pmtl_freq1.owner_oi ON MAINTENT2.mtnoi = PMTLACTIVITY1.specifiedent_oi FULL OUTER JOIN
    mc.PMJOB AS PMJOB1 LEFT OUTER JOIN
    mc.SITE AS SITE_1 ON PMJOB1.be_oi = SITE_1.siteoi FULL OUTER JOIN
    mc.EMPLOYEE AS EMPLOYEE1 FULL OUTER JOIN
    mc.PMTASK AS PMTASK1 LEFT OUTER JOIN
    mc.WORKORDERTYPE AS WORKORDERTYPE1 ON PMTASK1.wot_oi = WORKORDERTYPE1.wotyoi LEFT OUTER JOIN
    mc.PRIORITY AS PRIORITY1 ON PMTASK1.prio_oi = PRIORITY1.priooi LEFT OUTER JOIN
    mc.PMRESOURCE AS PMRESOURCE1 ON PMTASK1.pmtkoi = PMRESOURCE1.pmtsk_oi LEFT OUTER JOIN
    mc.PVMLABOURREQ AS PVMLABOURREQ1 ON PMTASK1.pmtkoi = PVMLABOURREQ1.pmtsk_oi LEFT OUTER JOIN
    mc.TRADE AS TRADE1 ON PVMLABOURREQ1.trd_oi = TRADE1.tradoi FULL OUTER JOIN
    mc.EMPLOYEE AS EMPLOYEE2 ON PMTASK1.plnr_oi = EMPLOYEE2.empoi ON EMPLOYEE1.empoi = PMTASK1.supr_oi ON PMJOB1.pmjboi = PMTASK1.pmjob_oi ON TSKLNK1.pmtsk_oi = PMTASK1.pmtkoi ON 
    MAINTENT1.mtnoi = TSKLNK1.mtn_oi
WHERE        (1 = 1) AND (TSKLNK1.ci_cnc_date == null) AND (PMJOB1.ci_cnc_date == null) AND (PMTASK1.ci_cnc_date == null) AND (pmtl_freq1.trigint != null) AND (pmtl_freq1.frqval_amt != null)
ORDER BY PM_JOB, PMTASK1.tskid"""


df = pd.read_sql(SQL1,engine)
df = df[~df['actdesc'].isna()]
df['ACT_ENTITY'] = df['ACT_ENTITY'].fillna(df['TASK_ENTITY'])
df

,PM_JOB,tskid,PM_TASK_NAME,TASK_ENTITY,TASK_ENTITY_NAME,TASK_ENTITY_TYPE,actseq,ACT_ENTITY,ACT_CLASS,actdesc
0,BOM-LAB,1,Laboratory - Pipette Calibration,COM-INORG-P,Pipettes - Inorganic,Equipment Location,2000000.0,LAB-P-285854,Pipette,Calibrate pipette LAB-P-285854
1,BOM-LAB,1,Laboratory - Pipette Calibration,COM-INORG-P,Pipettes - Inorganic,Equipment Location,1000000.0,LAB-P-10F94538,Pipette,Calibrate pipette LAB-P-10F94538
2,BOM-LAB,1,Laboratory - Pipette Calibration,COM-INORG-P,Pipettes - Inorganic,Equipment Location,3000000.0,LAB-P-3543442,Pipette,Calibrate pipette LAB-P-3543442
3,BOM-LAB,1,Laboratory - Pipette Calibration,COM-INORG-P,Pipettes - Inorganic,Equipment Location,5000000.0,LAB-P-J40896D,Pipette,Calibrate pipette LAB-P-J40896D
4,BOM-LAB,1,Laboratory - Pipette Calibration,COM-INORG-P,Pipettes - Inorganic,Equipment Location,4000000.0,LAB-P-63218,Pipette,Calibrate pipette LAB-P-63218
...,...,...,...,...,...,...,...,...,...,...
117256,WTR-SYP,3,Check and Calibrate System Pressure Transmitte...,WTR-SYP-PIT-042,"Transmitter,Pressure,System,District 4",Instrumentation Equipment,1000000.0,WTR-SYP-PIT-042,Transmitter,Check and Calibrate System Pressure Transmitte...
117257,WTR-SYP,3,Check and Calibrate System Pressure Transmitte...,WTR-SYP-PIT-036,"Transmitter,Pressure,System,District 3E",Instrumentation Equipment,1000000.0,WTR-SYP-PIT-036,Transmitter,Check and Calibrate System Pressure Transmitte...
117258,WTR-SYP,4,Monthly Flow Transmitter Flushing,WTR-PEL-FIT-0002,"Transmitter, Flow, Water to District 2E",Instrumentation Equipment,1000000.0,WTR-PEL-FIT-0002,None,Service the flow transmitter impulse line (WTR...
117261,YTF-SS-01,2,Monthly Inspections,YTF-SS,Security Systems,Work Area,2000000.0,YTF-SS-YAN-1000,Security System,Test the circuit for manually de-energizing an...


## Data cleaning

In [2]:
import re
# Regex pattern to match valid codes (hyphenated or not)
#ref_pattern = r'\b(?:[A-Z]{2,}(?:-[A-Z0-9]{2,})+|[A-Z]{2,}[0-9]+[A-Z0-9]*)\b'
ref_pattern = r'\b(?:[A-Z]{2,}(?:-[A-Z0-9]+)+|[A-Z]{2,}[0-9][A-Z0-9]*)\b'

# Pattern to match marker words: REF:, Ref:, REF#, Ref#, FOR:, For:
marker_pattern = r'\(?\b(?:REF[:.#]?|FOR:)\)?'

# List of known prefixes
prefixes = ['THC', 'TAB', 'THR', 'TNT', 'COL', 'DOS', 'WTR', 'FCL', 'FHA', 'FIS', 'FHO']
prefix_pattern = r'^(' + '|'.join(prefixes) + r')[-_]?'

# Structured reference pattern to EXCLUDE from 'ref' columns
structured_pattern = re.compile(
     rf'^({prefix_pattern})-[A-Z]{{2,4}}-[A-Z]{{1,4}}-\d{{2}}[A-Z0-9]{{2}}[A-Z]?$', re.IGNORECASE
)

# === Extract references and filter out structured ones ===

def extract_valid_refs(text):
    all_matches = re.findall(ref_pattern, text)
    # Exclude matches that look like structured references
    return [r for r in all_matches if not structured_pattern.match(r)]

df[['ref1', 'ref2', 'ref3']] = pd.DataFrame(
    df['actdesc'].apply(lambda x: (extract_valid_refs(x)[::-1][:3] + [None]*3)[:3]).tolist()
)

# Apply the same structured exclusion during string join
df['reference'] = df['actdesc'].apply(
    lambda text: ', '.join(
        r for r in re.findall(ref_pattern, text)
        if not structured_pattern.match(r)
    )
)

# === Clean actdesc ===

df['actdesc_cleaned'] = (
    df['actdesc']
    .str.replace(ref_pattern, '', regex=True)
    .str.replace(marker_pattern, '', flags=re.IGNORECASE, regex=True)
    .str.replace(r'[^\w\s]+', '', regex=True)         # remove punctuation: & ( ) etc.
    .str.replace(r'\s+', ' ', regex=True)             # normalize spacing
    .str.strip()
    .str.replace(r'[\W_]+$', '', regex=True)          # remove trailing punctuation if any
)

def standardize_reference(ref):
    if pd.isna(ref):
        return None
    # Remove corporate prefix if present
    ref = re.sub(prefix_pattern, '', ref, flags=re.IGNORECASE)
    # Remove all dashes/underscores
    ref = re.sub(r'[-_]', '', ref)
    return ref.upper()

# Apply to each reference column
for col in ['ref1', 'ref2', 'ref3']:
    df[col] = df[col].apply(standardize_reference)

df

,PM_JOB,tskid,PM_TASK_NAME,TASK_ENTITY,TASK_ENTITY_NAME,TASK_ENTITY_TYPE,actseq,ACT_ENTITY,ACT_CLASS,actdesc,ref1,ref2,ref3,reference,actdesc_cleaned
0,BOM-LAB,1,Laboratory - Pipette Calibration,COM-INORG-P,Pipettes - Inorganic,Equipment Location,2000000.0,LAB-P-285854,Pipette,Calibrate pipette LAB-P-285854,LABP285854,None,None,LAB-P-285854,Calibrate pipette
1,BOM-LAB,1,Laboratory - Pipette Calibration,COM-INORG-P,Pipettes - Inorganic,Equipment Location,1000000.0,LAB-P-10F94538,Pipette,Calibrate pipette LAB-P-10F94538,LABP10F94538,None,None,LAB-P-10F94538,Calibrate pipette
2,BOM-LAB,1,Laboratory - Pipette Calibration,COM-INORG-P,Pipettes - Inorganic,Equipment Location,3000000.0,LAB-P-3543442,Pipette,Calibrate pipette LAB-P-3543442,LABP3543442,None,None,LAB-P-3543442,Calibrate pipette
3,BOM-LAB,1,Laboratory - Pipette Calibration,COM-INORG-P,Pipettes - Inorganic,Equipment Location,5000000.0,LAB-P-J40896D,Pipette,Calibrate pipette LAB-P-J40896D,LABPJ40896D,None,None,LAB-P-J40896D,Calibrate pipette
4,BOM-LAB,1,Laboratory - Pipette Calibration,COM-INORG-P,Pipettes - Inorganic,Equipment Location,4000000.0,LAB-P-63218,Pipette,Calibrate pipette LAB-P-63218,LABP63218,None,None,LAB-P-63218,Calibrate pipette
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117256,WTR-SYP,3,Check and Calibrate System Pressure Transmitte...,WTR-SYP-PIT-042,"Transmitter,Pressure,System,District 4",Instrumentation Equipment,1000000.0,WTR-SYP-PIT-042,Transmitter,Check and Calibrate System Pressure Transmitte...,None,None,None,WTR-SYP01,Check and Calibrate System Pressure Transmitter
117257,WTR-SYP,3,Check and Calibrate System Pressure Transmitte...,WTR-SYP-PIT-036,"Transmitter,Pressure,System,District 3E",Instrumentation Equipment,1000000.0,WTR-SYP-PIT-036,Transmitter,Check and Calibrate System Pressure Transmitte...,None,None,None,WTR-SYP01,Check and Calibrate System Pressure Transmitter
117258,WTR-SYP,4,Monthly Flow Transmitter Flushing,WTR-PEL-FIT-0002,"Transmitter, Flow, Water to District 2E",Instrumentation Equipment,1000000.0,WTR-PEL-FIT-0002,None,Service the flow transmitter impulse line (WTR...,None,None,None,WTR-47A5,Service the flow transmitter impulse line
117261,YTF-SS-01,2,Monthly Inspections,YTF-SS,Security Systems,Work Area,2000000.0,YTF-SS-YAN-1000,Security System,Test the circuit for manually de-energizing an...,None,None,None,SS-30A1,Test the circuit for manually deenergizing and...


## Route finding

In [7]:
df = df[['PM_JOB','tskid','PM_TASK_NAME','TASK_ENTITY_NAME', 'TASK_ENTITY','actseq','ACT_ENTITY','actdesc_cleaned']]

groups = df.groupby(['PM_JOB','tskid','PM_TASK_NAME','TASK_ENTITY_NAME','TASK_ENTITY'])['actdesc_cleaned'].transform('nunique') == 1
many = df.groupby(['PM_JOB','tskid','PM_TASK_NAME','TASK_ENTITY_NAME','TASK_ENTITY'])['ACT_ENTITY'].transform('count') > 1
routes = df[(groups) & (many)].sort_values(['PM_JOB','tskid','PM_TASK_NAME','TASK_ENTITY'])
routes.groupby(['PM_JOB','tskid','PM_TASK_NAME','TASK_ENTITY_NAME','TASK_ENTITY']).sum()
routes.to_excel('routes.xlsx', index = False)
routes


,PM_JOB,tskid,PM_TASK_NAME,TASK_ENTITY_NAME,TASK_ENTITY,actseq,ACT_ENTITY,actdesc_cleaned
29,BOM-LAB,1,Laboratory - Pipette Calibration,Pipettes - Inorganic,TAB-INORG-P,2000000.0,LAB-P-229000Z,Calibrate pipette
31,BOM-LAB,1,Laboratory - Pipette Calibration,Pipettes - Inorganic,TAB-INORG-P,5000000.0,LAB-P-338770,Calibrate pipette
35,BOM-LAB,1,Laboratory - Pipette Calibration,Pipettes - Inorganic,TAB-INORG-P,1000000.0,LAB-P-1684268,Calibrate pipette
36,BOM-LAB,1,Laboratory - Pipette Calibration,Pipettes - Inorganic,TAB-INORG-P,3000000.0,LAB-P-2492863,Calibrate pipette
37,BOM-LAB,1,Laboratory - Pipette Calibration,Pipettes - Inorganic,TAB-INORG-P,4000000.0,LAB-P-317150,Calibrate pipette
...,...,...,...,...,...,...,...,...
117188,WTR-SES-01,5,Fall Arrest Equipment Inspection PM,Safety & Lifting Devices,WTR-WEST-SES,4000000.0,WTR-PLW-HRN-0004,Inspect personal safety harness by 3rd party c...
117189,WTR-SES-01,5,Fall Arrest Equipment Inspection PM,Safety & Lifting Devices,WTR-WEST-SES,7000000.0,WTR-PLW-HRN-0007,Inspect personal safety harness by 3rd party c...
117190,WTR-SES-01,5,Fall Arrest Equipment Inspection PM,Safety & Lifting Devices,WTR-WEST-SES,8000000.0,WTR-PLW-HRN-0008,Inspect personal safety harness by 3rd party c...
117191,WTR-SES-01,5,Fall Arrest Equipment Inspection PM,Safety & Lifting Devices,WTR-WEST-SES,11000000.0,WTR-PLW-HRN-0011,Inspect personal safety harness by 3rd party c...
